# xlwings 笔记

In [17]:
import xlwings as xw
import datetime

In [37]:
from IPython.display import display_pretty, display_html

## 默认转换器

xlwings 中的转换器定义了Excel 单元格和它们的值在读、写操作中的转换规则。
``` python
import xlwings as xw
xw.Range('A1:C3').options(pd.DataFrame, index=False, numbers=int).value
```
如果没有 `options`，则按照下面规则转换：
* 当读取单个单元格时：

|内容|转换|
|:--:|--|
|number|`float`|
|text| `str`|
|日期| `datetime` |
|空|`None`|

* 当读取行/列时，作为 `list` 读取，如： `[None, 1.0, 'a string']`
* 当读取范围大于一列/行时，则作为二维 `list` 读取，如：`[[None, 1.0, 'a string'], [None, 2.0, 'another string']]`

有如下选项可设置：

* `ndim` -- 不管 `range()` 中参数是单个单元格、1列/行、多列/行，强制返回值为一维或二维。

In [68]:
book = xw.Book()
sht = book.sheets[0]
sht.range('A1').value = [[1, 2], [3, 4]]
vlist = (v1,v2,v3,v4,v5 )= (sht.range('A1').value,
    sht.range('A1').options(ndim=1).value,
    sht.range('A1').options(ndim=2).value,
    sht.range('A1:A2').value,
    sht.range('A1:A2').options(ndim=2).value)
for i, v in enumerate(vlist):
    print('v%s: %s' % (str(i+1),v))
book.close()

v1: 1.0
v2: [1.0]
v3: [[1.0]]
v4: [1.0, 3.0]
v5: [[1.0], [3.0]]


* `numbers` -- 单元格数字默认读取为 `float` 类型，可设置为 `int` 类型。

In [64]:
book = xw.Book()
sht = book.sheets[0]
sht.range('D1').value = 1
v1 = sht.range('D1').value
v2 = sht.range('D1').options(numbers=int).value
print('v1: %s\nv2: %s' % (v1,v2))
book.close()

v1: 1.0
v2: 1


* `dates` -- 单元格日期默认读取为 `datetime.datetime`，但你可以改变为 `datetime.date`。

* `empty` -- 空单元格默认转换为 `None`，可改变为其他，如：`sht.range('A1').options(empty='NA').value`。

* `transpose` -- 按照列写入列表。

In [69]:
book = xw.Book()
sht = book.sheets[0]
sht.range('A1').options(transpose=True).value = [[1,2,3],[4,5,6]]
v1 = sht.range('A1:B3').value
v2 = sht.range('A1:B3').options(transpose=True).value
print(v1)
print(v2)
book.close()

[[1.0, 4.0], [2.0, 5.0], [3.0, 6.0]]
[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]


* `expand` -- 以 `table`、`vertical`、`horizontal`方式获取单元格范围的值。

In [73]:
book = xw.Book()
sht = book.sheets[0]
sht.range('A1').value = [[1,2], [3,4]]
rng1 = sht.range('A1').expand()
rng2 = sht.range('A1').options(expand='table')
print(rng1.value)
print(rng2.value)
print('='*30)
sht.range('A3').value = [5, 6]
print(rng1.value)
print(rng2.value)
print(sht.range('A1').options(expand='vertical').value)
book.close()

[[1.0, 2.0], [3.0, 4.0]]
[[1.0, 2.0], [3.0, 4.0]]
[[1.0, 2.0], [3.0, 4.0]]
[[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]]
[1.0, 3.0, 5.0]


## 内置转换器

###  Dictionary converter
将 Excel 的两列单元格转换为字典，如果需要按照行转换，则用 `transpose`参数：

In [77]:
book = xw.Book()
sht = book.sheets[0]
sht.range('A1').value = [['a',2], ['b',4]]
v1 = sht.range('A1:B2').options(dict).value
print(v1)
v2 = sht.range('A1:B2').options(dict, transpose=True, numbers=int).value
print(v2)

import time
time.sleep(10)
book.close()

{'a': 2.0, 'b': 4.0}
{2: 4, 'a': 'b'}


###  Numpy array converter

### Pandas Series converter

### Pandas DataFrame converter

### xw.Range and ‘raw’ converters

## Custom Converter

看完整的例子：

In [78]:
book = xw.Book('data/test.xlsx')

sht = book.sheets[0]

In [79]:
'''
url: http://calebmadrigal.com/display-list-as-table-in-ipython-notebook/
'''
class ListTable(list):
    """ Overridden list class which takes a 2-dimensional list of 
        the form [[1,2,3],[4,5,6]], and renders an HTML Table in 
        IPython Notebook. """
    
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            
            for col in row:
                html.append("<td>{0}</td>".format(col))
            
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)

In [80]:
ls = [
 ['日期',datetime.datetime(2015, 1, 1, 0, 0),datetime.datetime(2015, 9, 20, 0, 0)],
 ['数字', 1.0, 1.10],
 ['常规', 1.0, 1/3],
 ['百分比', 1.1, 0.5],
 ['文本', '中文', '11.10']
]

sht.range('A1').value = ls

rng = sht.range('A1').current_region

In [81]:
display_html(ListTable(rng.value))
print(*rng.value, sep='\n')

日期,2015-01-01 00:00:00,2015-09-20 00:00:00,1.0
数字,1.0,1.1,None
常规,1.0,0.3333333333333333,None
百分比,1.1,0.5,None
文本,中文,11.10,None


['日期', datetime.datetime(2015, 1, 1, 0, 0), datetime.datetime(2015, 9, 20, 0, 0), 1.0]
['数字', 1.0, 1.1, None]
['常规', 1.0, 0.3333333333333333, None]
['百分比', 1.1, 0.5, None]
['文本', '中文', '11.10', None]
